# Réseaux de neurones récurrents

Dans le module précédent, nous avons utilisé des représentations sémantiques riches de texte, associées à un simple classificateur linéaire au-dessus des embeddings. Cette architecture permet de capturer le sens global des mots dans une phrase, mais elle ne prend pas en compte l'**ordre** des mots, car l'opération d'agrégation appliquée aux embeddings supprime cette information issue du texte original. Étant donné que ces modèles ne peuvent pas modéliser l'ordre des mots, ils ne sont pas capables de résoudre des tâches plus complexes ou ambiguës, comme la génération de texte ou la réponse à des questions.

Pour capturer le sens d'une séquence de texte, nous devons utiliser une autre architecture de réseau de neurones, appelée **réseau de neurones récurrent**, ou RNN. Dans un RNN, nous faisons passer notre phrase à travers le réseau, un symbole à la fois, et le réseau produit un certain **état**, que nous transmettons ensuite au réseau avec le symbole suivant.

Étant donné la séquence d'entrée de tokens $X_0,\dots,X_n$, le RNN crée une séquence de blocs de réseau de neurones et entraîne cette séquence de bout en bout à l'aide de la rétropropagation. Chaque bloc de réseau prend une paire $(X_i,S_i)$ en entrée et produit $S_{i+1}$ en sortie. L'état final $S_n$ ou la sortie $X_n$ est ensuite transmis à un classificateur linéaire pour produire le résultat. Tous les blocs de réseau partagent les mêmes poids et sont entraînés de bout en bout en une seule passe de rétropropagation.

Grâce aux vecteurs d'état $S_0,\dots,S_n$ qui sont transmis à travers le réseau, celui-ci est capable d'apprendre les dépendances séquentielles entre les mots. Par exemple, lorsque le mot *pas* apparaît quelque part dans la séquence, le réseau peut apprendre à inverser certains éléments du vecteur d'état, ce qui entraîne une négation.

> Étant donné que les poids de tous les blocs RNN sur l'image sont partagés, la même image peut être représentée par un seul bloc (à droite) avec une boucle de rétroaction récurrente, qui renvoie l'état de sortie du réseau à l'entrée.

Voyons comment les réseaux de neurones récurrents peuvent nous aider à classifier notre ensemble de données de nouvelles.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Classificateur RNN simple

Dans le cas d'un RNN simple, chaque unité récurrente est un réseau linéaire simple, qui prend un vecteur d'entrée concaténé et un vecteur d'état, et produit un nouveau vecteur d'état. PyTorch représente cette unité avec la classe `RNNCell`, et un réseau de telles cellules - comme une couche `RNN`.

Pour définir un classificateur RNN, nous appliquerons d'abord une couche d'embedding pour réduire la dimensionnalité du vocabulaire d'entrée, puis ajouterons une couche RNN par-dessus :


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** Nous utilisons ici une couche d'embedding non entraînée pour simplifier, mais pour obtenir de meilleurs résultats, nous pouvons utiliser une couche d'embedding pré-entraînée avec des embeddings Word2Vec ou GloVe, comme décrit dans l'unité précédente. Pour mieux comprendre, vous pourriez adapter ce code pour qu'il fonctionne avec des embeddings pré-entraînés.

Dans notre cas, nous utiliserons un chargeur de données avec padding, de sorte que chaque lot contiendra un certain nombre de séquences remplies pour avoir la même longueur. La couche RNN prendra la séquence de tenseurs d'embedding et produira deux sorties :  
* $x$ est une séquence des sorties des cellules RNN à chaque étape  
* $h$ est l'état caché final pour le dernier élément de la séquence  

Nous appliquons ensuite un classificateur linéaire entièrement connecté pour obtenir le nombre de classes.

> **Note:** Les RNN sont assez difficiles à entraîner, car une fois que les cellules RNN sont déroulées sur la longueur de la séquence, le nombre de couches impliquées dans la rétropropagation devient assez important. Par conséquent, nous devons sélectionner un faible taux d'apprentissage et entraîner le réseau sur un ensemble de données plus large pour obtenir de bons résultats. Cela peut prendre beaucoup de temps, donc l'utilisation d'un GPU est préférable.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Mémoire à Long et Court Terme (LSTM)

L'un des principaux problèmes des RNN classiques est le problème des **gradients qui disparaissent**. Étant donné que les RNN sont entraînés de bout en bout en une seule passe de rétropropagation, il est difficile de propager l'erreur jusqu'aux premières couches du réseau, ce qui empêche le réseau d'apprendre les relations entre des tokens éloignés. Une des façons de contourner ce problème est d'introduire une **gestion explicite de l'état** en utilisant ce qu'on appelle des **portes**. Les deux architectures les plus connues de ce type sont : **Mémoire à Long et Court Terme** (LSTM) et **Unité de Relais Gâtée** (GRU).

![Image montrant un exemple de cellule de mémoire à long et court terme](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Le réseau LSTM est organisé de manière similaire au RNN, mais il y a deux états qui sont transmis d'une couche à l'autre : l'état actuel $c$, et le vecteur caché $h$. À chaque unité, le vecteur caché $h_i$ est concaténé avec l'entrée $x_i$, et ils contrôlent ce qui arrive à l'état $c$ via des **portes**. Chaque porte est un réseau neuronal avec une activation sigmoïde (sortie dans la plage $[0,1]$), qui peut être considérée comme un masque binaire lorsqu'elle est multipliée par le vecteur d'état. Les portes suivantes existent (de gauche à droite sur l'image ci-dessus) :
* **Porte d'oubli** : prend le vecteur caché et détermine quelles composantes du vecteur $c$ doivent être oubliées et lesquelles doivent être conservées.
* **Porte d'entrée** : prend certaines informations de l'entrée et du vecteur caché, et les insère dans l'état.
* **Porte de sortie** : transforme l'état via une couche linéaire avec activation $\tanh$, puis sélectionne certaines de ses composantes en utilisant le vecteur caché $h_i$ pour produire le nouvel état $c_{i+1}$.

Les composantes de l'état $c$ peuvent être considérées comme des indicateurs qui peuvent être activés ou désactivés. Par exemple, lorsque nous rencontrons un nom comme *Alice* dans une séquence, nous pouvons supposer qu'il fait référence à un personnage féminin et activer l'indicateur dans l'état indiquant que nous avons un nom féminin dans la phrase. Lorsque nous rencontrons ensuite des expressions comme *et Tom*, nous activons l'indicateur indiquant que nous avons un nom au pluriel. Ainsi, en manipulant l'état, nous pouvons théoriquement suivre les propriétés grammaticales des parties de la phrase.

> **Note** : Une excellente ressource pour comprendre les détails des LSTM est cet article [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) de Christopher Olah.

Bien que la structure interne d'une cellule LSTM puisse sembler complexe, PyTorch cache cette implémentation dans la classe `LSTMCell` et fournit l'objet `LSTM` pour représenter toute la couche LSTM. Ainsi, l'implémentation d'un classificateur LSTM sera assez similaire à celle du RNN simple que nous avons vu précédemment :


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Séquences compactées

Dans notre exemple, nous avons dû compléter toutes les séquences du minibatch avec des vecteurs de zéros. Bien que cela entraîne un certain gaspillage de mémoire, avec les RNN, il est encore plus problématique que des cellules RNN supplémentaires soient créées pour les éléments d'entrée complétés, qui participent à l'entraînement mais ne contiennent aucune information d'entrée importante. Il serait bien mieux d'entraîner le RNN uniquement sur la taille réelle des séquences.

Pour cela, un format spécial de stockage des séquences complétées est introduit dans PyTorch. Supposons que nous ayons un minibatch complété qui ressemble à ceci :  
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```  
Ici, 0 représente les valeurs complétées, et le vecteur des longueurs réelles des séquences d'entrée est `[5,3,1]`.

Pour entraîner efficacement un RNN avec des séquences complétées, nous souhaitons commencer l'entraînement du premier groupe de cellules RNN avec un grand minibatch (`[1,6,9]`), mais ensuite arrêter le traitement de la troisième séquence et continuer l'entraînement avec des minibatches réduits (`[2,7]`, `[3,8]`), et ainsi de suite. Ainsi, une séquence compactée est représentée comme un seul vecteur - dans notre cas `[1,6,9,2,7,3,8,4,5]`, et un vecteur de longueurs (`[5,3,1]`), à partir duquel nous pouvons facilement reconstruire le minibatch complété d'origine.

Pour produire une séquence compactée, nous pouvons utiliser la fonction `torch.nn.utils.rnn.pack_padded_sequence`. Toutes les couches récurrentes, y compris RNN, LSTM et GRU, prennent en charge les séquences compactées en tant qu'entrée et produisent une sortie compactée, qui peut être décodée à l'aide de `torch.nn.utils.rnn.pad_packed_sequence`.

Pour pouvoir produire une séquence compactée, nous devons transmettre le vecteur des longueurs au réseau, et donc nous avons besoin d'une fonction différente pour préparer les minibatches :


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Le réseau réel serait très similaire à `LSTMClassifier` ci-dessus, mais le passage `forward` recevra à la fois le mini-lot avec remplissage et le vecteur des longueurs de séquence. Après avoir calculé l'embedding, nous calculons la séquence empaquetée, la passons à la couche LSTM, puis dépaquetons le résultat.

> **Note** : En réalité, nous n'utilisons pas le résultat dépaqueté `x`, car nous utilisons la sortie des couches cachées dans les calculs suivants. Ainsi, nous pouvons supprimer complètement le dépaquetage de ce code. La raison pour laquelle nous le plaçons ici est de vous permettre de modifier ce code facilement, au cas où vous auriez besoin d'utiliser la sortie du réseau dans des calculs ultérieurs.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Remarque :** Vous avez peut-être remarqué le paramètre `use_pack_sequence` que nous passons à la fonction d'entraînement. Actuellement, la fonction `pack_padded_sequence` nécessite que le tenseur de séquence de longueur soit sur le périphérique CPU, et donc la fonction d'entraînement doit éviter de déplacer les données de séquence de longueur vers le GPU lors de l'entraînement. Vous pouvez consulter l'implémentation de la fonction `train_emb` dans le fichier [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## RNN bidirectionnels et multicouches

Dans nos exemples, tous les réseaux récurrents fonctionnaient dans une seule direction, de l'origine d'une séquence jusqu'à sa fin. Cela semble naturel, car cela ressemble à la manière dont nous lisons et écoutons un discours. Cependant, dans de nombreux cas pratiques où nous avons un accès aléatoire à la séquence d'entrée, il peut être pertinent d'exécuter le calcul récurrent dans les deux directions. Ces réseaux sont appelés **RNN bidirectionnels**, et ils peuvent être créés en passant le paramètre `bidirectional=True` au constructeur RNN/LSTM/GRU.

Lorsqu'on travaille avec un réseau bidirectionnel, il nous faut deux vecteurs d'état caché, un pour chaque direction. PyTorch encode ces vecteurs en un seul vecteur de taille double, ce qui est assez pratique, car on passe généralement l'état caché résultant à une couche linéaire entièrement connectée, et il suffit de prendre en compte cette augmentation de taille lors de la création de la couche.

Un réseau récurrent, qu'il soit unidirectionnel ou bidirectionnel, capture certains motifs au sein d'une séquence et peut les stocker dans un vecteur d'état ou les transmettre en sortie. Comme pour les réseaux convolutionnels, on peut construire une autre couche récurrente au-dessus de la première pour capturer des motifs de niveau supérieur, construits à partir des motifs de bas niveau extraits par la première couche. Cela nous amène à la notion de **RNN multicouche**, qui consiste en deux ou plusieurs réseaux récurrents, où la sortie de la couche précédente est transmise à la couche suivante comme entrée.

![Image montrant un RNN multicouche avec mémoire à court et long terme](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Image tirée de [cet excellent article](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) par Fernando López*

PyTorch simplifie la construction de tels réseaux, car il suffit de passer le paramètre `num_layers` au constructeur RNN/LSTM/GRU pour créer automatiquement plusieurs couches de récurrence. Cela signifie également que la taille du vecteur caché/d'état augmente proportionnellement, et il faut en tenir compte lors de la gestion de la sortie des couches récurrentes.


## RNNs pour d'autres tâches

Dans cette unité, nous avons vu que les RNNs peuvent être utilisés pour la classification de séquences, mais en réalité, ils peuvent gérer bien d'autres tâches, comme la génération de texte, la traduction automatique, et bien plus encore. Nous aborderons ces tâches dans la prochaine unité.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
